# COVID-19 Data Analysis - Maryland Dataset

This notebook performs data preparation and analysis on the Maryland COVID-19 dataset. The analysis includes:
1. Data loading and cleaning
2. Data reshaping from wide to long format
3. Clustering analysis using K-means
4. Data export to CSV

# 1. Import Required Libraries and Load Data

First, we'll import the necessary libraries and load our dataset:

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Read the COVID-19 dataset
df = pd.read_csv('data/MD_COVID_19_dataset.csv')

In [9]:
# Display unique date formats in the dataset
print("Sample of unique date values:")
print(df['DATE'].unique()[:10])

Sample of unique date values:
['4/29/2020 10:00' '5/6/2020 10:00' '5/13/2020 10:00' '5/20/2020 10:00'
 '5/27/2020 10:00' '6/3/2020 10:00' '6/10/2020 10:00' '6/17/2020 10:00'
 '6/24/2020 10:00' '7/1/2020 10:00']


In [5]:
# Display the first few rows to check the date format
print("Date column example values:")
print(df['DATE'].head())
print("\nDataframe columns:")
print(df.columns.tolist())

Date column example values:
0    4/29/2020 10:00
1    4/29/2020 10:00
2    4/29/2020 10:00
3    4/29/2020 10:00
4    4/29/2020 10:00
Name: DATE, dtype: object

Dataframe columns:
['OBJECTID', 'DATE', 'Facility_Type', 'Allegany', 'Anne_Arundel', 'Baltimore', 'Baltimore_City', 'Calvert', 'Caroline', 'Carroll', 'Cecil', 'Charles', 'Dorchester', 'Frederick', 'Garrett', 'Harford', 'Howard', 'Kent', 'Montgomery', 'Prince_George_s', 'Queen_Anne_s', 'Somerset', 'St_Mary_s', 'Talbot', 'Washington', 'Wicomico', 'Worcester']


# 2. Data Cleaning and Date Conversion

Now we'll clean the data by:
1. Converting the DATE column to datetime
2. Identifying county-specific columns
3. Filling missing values with 0

In [12]:
# Convert DATE column to datetime
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')

# Identify county columns (all columns between 'Allegany' and 'Worcester')
county_cols = df.loc[:, 'Allegany':'Worcester'].columns

# Fill missing values with 0 in county columns
df[county_cols] = df[county_cols].fillna(0)

# Display the first few rows of the cleaned dataset
print("Dataset shape:", df.shape)
print("\nFirst few rows after cleaning:")
df.head()

Dataset shape: (732, 27)

First few rows after cleaning:


,OBJECTID,DATE,Facility_Type,Allegany,Anne_Arundel,Baltimore,Baltimore_City,Calvert,Caroline,Carroll,...,Kent,Montgomery,Prince_George_s,Queen_Anne_s,Somerset,St_Mary_s,Talbot,Washington,Wicomico,Worcester
0,55,2020-04-29 10:00:00,"Staff: Nursing, Assisted Living, Group Homes A...",17,143,232,172,2,0,69,...,10,382,173,3,0,7,0,6,2,3
1,56,2020-04-29 10:00:00,"Residents: Nursing, Assisted Living, Group Hom...",73,273,526,490,0,0,203,...,57,717,296,0,0,12,0,6,6,3
2,57,2020-04-29 10:00:00,Staff: State and Local Affected Facilities,3,133,13,66,0,0,3,...,0,3,31,0,1,0,0,2,1,0
3,58,2020-04-29 10:00:00,Patients: State and Local Affected Facilities,0,0,28,0,0,0,0,...,0,0,19,0,0,0,0,0,0,0
4,59,2020-04-29 10:00:00,Inmates: State and Local Affected Facilities,0,52,0,4,0,0,0,...,0,0,0,0,0,0,0,1,0,0


# 3. Data Reshaping (Wide to Long Format)

Next, we'll transform the data from wide format to long format using the pandas melt function:

In [13]:
# Reshape the data from wide to long format
df_long = pd.melt(
    df,
    id_vars=['DATE', 'Facility_Type'],
    value_vars=county_cols,
    var_name='County',
    value_name='Case_Count'
)

# Display the first few rows of the reshaped dataset
print("Long format dataset shape:", df_long.shape)
df_long.head()

Long format dataset shape: (17568, 4)


,DATE,Facility_Type,County,Case_Count
0,2020-04-29 10:00:00,"Staff: Nursing, Assisted Living, Group Homes A...",Allegany,17
1,2020-04-29 10:00:00,"Residents: Nursing, Assisted Living, Group Hom...",Allegany,73
2,2020-04-29 10:00:00,Staff: State and Local Affected Facilities,Allegany,3
3,2020-04-29 10:00:00,Patients: State and Local Affected Facilities,Allegany,0
4,2020-04-29 10:00:00,Inmates: State and Local Affected Facilities,Allegany,0


# 4. Clustering Analysis

Now we'll use K-means clustering to group counties based on their total case counts:

In [14]:
# Create summary dataframe with total cases by county
county_summary = df_long.groupby('County')['Case_Count'].sum().reset_index()

# Prepare data for clustering
X = county_summary[['Case_Count']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
county_summary['Cluster'] = kmeans.fit_predict(X_scaled)

# Add meaningful cluster labels
cluster_means = county_summary.groupby('Cluster')['Case_Count'].mean()
cluster_order = cluster_means.sort_values().index
label_map = {
    cluster_order[0]: 'Low Incident',
    cluster_order[1]: 'Medium Incident',
    cluster_order[2]: 'High Incident'
}
county_summary['Cluster_Label'] = county_summary['Cluster'].map(label_map)

# Sort by total cases and display results
county_summary = county_summary.sort_values('Case_Count', ascending=False)
print("\nCounty Clustering Results:")
print(county_summary[['County', 'Case_Count', 'Cluster_Label']])


County Clustering Results:
             County  Case_Count    Cluster_Label
2         Baltimore      348546    High Incident
3    Baltimore_City      275689    High Incident
15       Montgomery      215433    High Incident
1      Anne_Arundel      147075  Medium Incident
16  Prince_George_s      103584  Medium Incident
21       Washington       90305  Medium Incident
0          Allegany       64503     Low Incident
10        Frederick       57010     Low Incident
13           Howard       55111     Low Incident
6           Carroll       51644     Low Incident
7             Cecil       47159     Low Incident
22         Wicomico       35013     Low Incident
12          Harford       34500     Low Incident
23        Worcester       16813     Low Incident
8           Charles       16356     Low Incident
11          Garrett       13377     Low Incident
9        Dorchester        9775     Low Incident
4           Calvert        7419     Low Incident
5          Caroline        6549     Low I

# 5. Data Export

Finally, we'll save the cleaned, long-format data to a CSV file:

In [15]:
# Save the long-format dataframe to CSV
df_long.to_csv('../data/cleaned_covid_data.csv', index=False)
print("Data saved successfully to 'data/cleaned_covid_data.csv'")

Data saved successfully to 'data/cleaned_covid_data.csv'
